In [4]:
import pandas as pd
import numpy as np


In [5]:
df=pd.read_csv("/Users/antoniojaenarias/Desktop/DATA SCIENCE/TODO/ML_KAGGLE_1/data/data.csv")
df

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8
...,...,...,...,...,...,...,...,...,...,...
31639,31639,Thompson,NaN,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3
31640,31640,NaN,NaN,13.79,NaN,NaN,4.7,4.7,NaN,3.9
31641,31641,Thompson,NaN,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3
31642,31642,Thompson,tube,6.89,3825.0,NaN,23.6,23.6,1972.0,3.7


In [6]:
train=df[df['x_e_out [-]'].isnull()!=True]
train.reset_index(inplace=True,drop=True)

test=df[df['x_e_out [-]'].isnull()==True]
test.reset_index(inplace=True,drop=True)
test

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8
1,7,Peskov,tube,18.00,750.0,NaN,10.0,10.0,1650.0,2.2
2,10,Thompson,tube,NaN,NaN,NaN,1.9,1.9,152.0,3.2
3,12,Thompson,NaN,6.89,7500.0,NaN,NaN,12.8,1930.0,4.8
4,23,Beus,annulus,15.51,1355.0,NaN,5.6,15.2,2134.0,2.1
...,...,...,...,...,...,...,...,...,...,...
10410,31633,Thompson,tube,11.03,NaN,NaN,11.5,11.5,NaN,2.0
10411,31634,Richenderfer,plate,1.01,2000.0,NaN,15.0,120.0,10.0,6.2
10412,31637,Weatherhead,tube,13.79,688.0,NaN,NaN,11.1,457.0,2.3
10413,31640,NaN,NaN,13.79,NaN,NaN,4.7,4.7,NaN,3.9


In [8]:
num_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
num_cols.remove('id')

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

all_features=num_cols + cat_cols

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in cat_cols:
        train[col] = label_encoder.fit_transform(train[col].astype(str))
        test[col] = label_encoder.transform(test[col].astype(str))
train["author"]

/Users/antoniojaenarias/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/Users/antoniojaenarias/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


0         7
1         7
2         7
3         0
4        10
         ..
21224    10
21225     7
21226     7
21227     7
21228    10
Name: author, Length: 21229, dtype: int64

In [9]:
X=train.drop('x_e_out [-]', axis=1)
y=train["x_e_out [-]"]

print(X.shape)
print(y.shape)

(21229, 9)
(21229,)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=1,
                                                    shuffle=True)

print("TRAIN SIZE:", X_train.shape)
print("TEST SIZE:", X_test.shape)

print("TRAIN TARGET SIZE:", y_train.shape)
print("TEST TARGET SIZE:", y_test.shape)

TRAIN SIZE: (16983, 9)
TEST SIZE: (4246, 9)
TRAIN TARGET SIZE: (16983,)
TEST TARGET SIZE: (4246,)


In [11]:
from sklearn.impute import KNNImputer

# Crea una instancia del imputador KNN
imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Rellena los valores nulos en X_train
X_train_imputed = imputer.fit_transform(X_train)

# Rellena los valores nulos en X_test
X_test_imputed = imputer.transform(X_test)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

model=LinearRegression()
model.fit(X_train_imputed, y_train)
y_pred=model.predict(X_test_imputed)

mse=mean_squared_error(y_test, y_pred)
rmse=sqrt(mse)

print("RMSE:", rmse)


RMSE: 0.08835702927749432


In [17]:
# Separar las características (X) y la columna objetivo (y)
X_nuevos_datos = test.drop(columns=["x_e_out [-]"])
y_nuevos_datos = test["x_e_out [-]"]
# Imputar los valores nulos en las características (X)
imputer = KNNImputer(n_neighbors=5)
X_nuevos_datos_imputed = imputer.fit_transform(X_nuevos_datos)
# Convertir el resultado a DataFrame
X_nuevos_datos_imputed = pd.DataFrame(X_nuevos_datos_imputed, columns=X_nuevos_datos.columns)
# Crear el objeto de regresión lineal
regression_model = LinearRegression()
# Entrenar el modelo de regresión lineal
regression_model.fit(X_train_imputed, y_train)
# Realizar predicciones en la columna objetivo (y) para los datos imputados
y_pred = regression_model.predict(X_nuevos_datos_imputed)
# Combinar las predicciones con los datos originales
test["x_e_out [-]"] = y_pred

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/Users/antoniojaenarias/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# Guardar los datos en un archivo para la competición de Kaggle
test.to_csv("nuevos_datos_con_predicciones.csv", index=False)
# Leer el archivo CSV
prueba = pd.read_csv("nuevos_datos_con_predicciones.csv")
# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
prueba = prueba[columnas_deseadas]
# Guardar el DataFrame resultante en un nuevo archivo CSV
prueba.to_csv("nuevos_datos_con_predicciones_filtrado.csv", index=False)